# Wows replay scraping notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import re
import json
import numpy as np
import requests
import time
import sys
import traceback


## Get the ship list

In [ ]:
r = requests.get('https://api.worldofwarships.eu/wows/encyclopedia/ships/?application_id=TOFILL&fields=ship_id%2C+name&page_no=1')


In [ ]:
initial_content = json.loads(r.content)
page_total = initial_content["meta"]["page_total"]

In [ ]:
ships = []
ships_name = []
data = initial_content["data"]
for elem in data:
    ships.append(elem)
    ships_name.append(data[elem]["name"])

In [ ]:
for i in range(2, page_total+1):
    r = requests.get(f'https://api.worldofwarships.eu/wows/encyclopedia/ships/?application_id=TOFILL&fields=ship_id%2C+name&page_no={i}')
    content = json.loads(r.content)
    data = content["data"]
    for elem in data:
        ships.append(elem)
        ships_name.append(data[elem]["name"])

In [ ]:
len(ships)

434

## Scrap a single replay

In [ ]:
data_train = np.empty((0,877), int)
time_table = []
for replay_nb in range(93100, 93100+1):
    timea = time.time()
    r = requests.get(f"https://replayswows.com/replay/{replay_nb}#stats")
    victory = 0
    if re.findall(b"DEFEAT", r.content):
        victory = 0
    elif re.findall(b"DRAW", r.content):
        continue
    else:
        victory = 1
    indicatif = ""
    if re.findall(b"\(NA\)", r.content):
        indicatif = "com"
    elif re.findall(b"\(EU\)", r.content):
        indicatif = "eu"
    elif re.findall(b"\(ASIA\)", r.content):
        indicatif = "asia"
    elif re.findall(b"\(RU\)", r.content):
        indicatif = "ru"
    else:
        continue
    r = requests.get(f"https://replayswows.com/replay/download/{replay_nb}")
    string = r.content.decode(errors='ignore')
    b = re.findall(r"\{[\x20-\x7D]*\}", string)
    to_json = b[0]
    try:
        jsonized = json.loads(to_json)
    except:
        continue
    vector1 = [0 for _ in range(len(ships))]
    vector2 = [0 for _ in range(len(ships))]
    ships = np.array(ships)
    player_names1, player_names2 = [], []
    player_id1, player_id2 = [], []
    player_wr1, player_wr2 = [], []
    player_exp1, player_exp2 = [], []
    for player in jsonized["vehicles"]:
        try:
          vec_number = np.where(ships==str(player["shipId"]))[0][0]
          if player["relation"] in [0, 1]:
              vector1[vec_number] += 1
          else:
              vector2[vec_number] += 1
        except:
          continue
        
        if player["relation"] in [0, 1]:
            if player["name"][0] == ':' or "IDS_OP" in player["name"]:
              continue
            player_names1.append(player["name"])
        else:
            if player["name"][0] == ':' or "IDS_OP" in player["name"]:
              continue
            player_names2.append(player["name"])
    if (not player_names1) or (not player_names2):
      continue
    requestable_names = "%2C".join(player_names1) + "%2C" + "%2C".join(player_names2)
    r = requests.get(f'https://api.worldofwarships.{indicatif}/wows/account/list/?search={requestable_names}&application_id=TOFILL&type=exact')
    player_dict = {}
    try:
      json.loads(r.content)["data"]
    except:
      print(json.loads(r.content))
      print(replay_nb)
    for account in json.loads(r.content)["data"]:
      player_dict[account["nickname"]] = account["account_id"]
    player_id1 = [str(player_dict.get(elem, None)) for elem in player_names1 if player_dict.get(elem, None)!= None]
    player_id2 = [str(player_dict.get(elem, None)) for elem in player_names2 if player_dict.get(elem, None)!= None]
    requestable_ids = "%2C".join(player_id1) + "%2C" + "%2C".join(player_id2)
    if (not player_id1) or (not player_id2):
      continue
    r = requests.get(f"https://api.worldofwarships.{indicatif}/wows/account/info/?application_id=TOFILL&account_id={requestable_ids}&fields=statistics.pvp.xp%2Cstatistics.pvp.battles%2Cstatistics.pvp.wins%2Cstatistics.pvp.losses")
    a = json.loads(r.content)["data"]
    player_exp1 = ([a[id]["statistics"]["pvp"]["xp"]/a[id]["statistics"]["pvp"]["battles"] for id in player_id1 if a[id]["statistics"]!=None and a[id]["statistics"]["pvp"]["battles"]!=0])
    player_wr1 = ([a[id]["statistics"]["pvp"]["wins"]/a[id]["statistics"]["pvp"]["battles"] for id in player_id1 if a[id]["statistics"]!=None and a[id]["statistics"]["pvp"]["battles"]!=0])
    player_exp2 = ([a[id]["statistics"]["pvp"]["xp"]/a[id]["statistics"]["pvp"]["battles"] for id in player_id2 if a[id]["statistics"]!=None and a[id]["statistics"]["pvp"]["battles"]!=0])
    player_wr2 = ([a[id]["statistics"]["pvp"]["wins"]/a[id]["statistics"]["pvp"]["battles"] for id in player_id2 if a[id]["statistics"]!=None and a[id]["statistics"]["pvp"]["battles"]!=0])

    train = vector1 + [np.mean(player_exp1), np.max(player_exp1), np.mean(player_wr1), np.max(player_wr2)] + vector2 + [np.mean(player_exp1), np.max(player_exp1), np.mean(player_wr1), np.max(player_wr2)] + [victory]
    timeb = - timea + time.time()
    time_table.append(timeb)
    data_train = np.vstack((data_train, train))


In [ ]:
np.mean(time_table)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

## Multithreaded version to scrap a lot of replays

In [ ]:
from queue  import Queue
from threading import Thread
import threading


In [ ]:
def get_data(borneinf, bornesup, queue, time_queue, ships):
  data_train = np.empty((0,877))
  for replay_nb in range(borneinf, bornesup+1):
      r = requests.get(f"https://replayswows.com/replay/{replay_nb}#stats")
      victory = 0
      if re.findall(b"DEFEAT", r.content):
          victory = 0
      elif re.findall(b"DRAW", r.content):
          continue
      else:
          victory = 1
      indicatif = ""
      if re.findall(b"\(NA\)", r.content):
          indicatif = "com"
      elif re.findall(b"\(EU\)", r.content):
          indicatif = "eu"
      elif re.findall(b"\(ASIA\)", r.content):
          indicatif = "asia"
      elif re.findall(b"\(RU\)", r.content):
          indicatif = "ru"
      else:
          continue
      r = requests.get(f"https://replayswows.com/replay/download/{replay_nb}")
      string = r.content.decode(errors='ignore')
      b = re.findall(r"\{[\x20-\x7D]*\}", string)
      to_json = b[0]
      try:
          jsonized = json.loads(to_json)
      except:
          continue
      vector1 = [0 for _ in range(len(ships))]
      vector2 = [0 for _ in range(len(ships))]
      ships = np.array(ships)
      player_names1, player_names2 = [], []
      player_id1, player_id2 = [], []
      player_wr1, player_wr2 = [], []
      player_exp1, player_exp2 = [], []
      for player in jsonized["vehicles"]:
          try:
            vec_number = np.where(ships==str(player["shipId"]))[0][0]
            if player["relation"] in [0, 1]:
                vector1[vec_number] += 1
            else:
                vector2[vec_number] += 1
          except:
            continue
          
          if player["relation"] in [0, 1]:
              if player["name"][0] == ':' or "IDS_OP" in player["name"]:
                continue
              player_names1.append(player["name"])
          else:
              if player["name"][0] == ':' or "IDS_OP" in player["name"]:
                continue
              player_names2.append(player["name"])
      if (not player_names1) or (not player_names2):
        continue
      requestable_names = "%2C".join(player_names1) + "%2C" + "%2C".join(player_names2)
      r = requests.get(f'https://api.worldofwarships.{indicatif}/wows/account/list/?search={requestable_names}&application_id=TOFILL&type=exact')
      player_dict = {}
      try:
        json.loads(r.content)["data"]
      except:
        print(json.loads(r.content))
        print(replay_nb)
      for account in json.loads(r.content)["data"]:
        player_dict[account["nickname"]] = account["account_id"]
      player_id1 = [str(player_dict.get(elem, None)) for elem in player_names1 if player_dict.get(elem, None)!= None]
      player_id2 = [str(player_dict.get(elem, None)) for elem in player_names2 if player_dict.get(elem, None)!= None]
      requestable_ids = "%2C".join(player_id1) + "%2C" + "%2C".join(player_id2)
      if (not player_id1) or (not player_id2):
        continue
      r = requests.get(f"https://api.worldofwarships.{indicatif}/wows/account/info/?application_id=TOFILL&account_id={requestable_ids}&fields=statistics.pvp.xp%2Cstatistics.pvp.battles%2Cstatistics.pvp.wins%2Cstatistics.pvp.losses")
      a = json.loads(r.content)["data"]
      player_exp1 = ([a[id]["statistics"]["pvp"]["xp"]/a[id]["statistics"]["pvp"]["battles"] for id in player_id1 if a[id]["statistics"]!=None and a[id]["statistics"]["pvp"]["battles"]!=0])
      player_wr1 = ([a[id]["statistics"]["pvp"]["wins"]/a[id]["statistics"]["pvp"]["battles"] for id in player_id1 if a[id]["statistics"]!=None and a[id]["statistics"]["pvp"]["battles"]!=0])
      player_exp2 = ([a[id]["statistics"]["pvp"]["xp"]/a[id]["statistics"]["pvp"]["battles"] for id in player_id2 if a[id]["statistics"]!=None and a[id]["statistics"]["pvp"]["battles"]!=0])
      player_wr2 = ([a[id]["statistics"]["pvp"]["wins"]/a[id]["statistics"]["pvp"]["battles"] for id in player_id2 if a[id]["statistics"]!=None and a[id]["statistics"]["pvp"]["battles"]!=0])
      if (not player_exp1) or (not player_exp2) or (not player_wr1) or (not player_wr2):
        continue
      train = vector1 + [np.mean(player_exp1), np.max(player_exp1), np.mean(player_wr1), np.max(player_wr1)] + vector2 + [np.mean(player_exp2), np.max(player_exp2), np.mean(player_wr2), np.max(player_wr2)] + [victory]
      data_train = np.vstack((data_train, train))
  queue.put(data_train)

class DataGetter(Thread):
    def __init__(self, id, borneinf, bornesup, queue, time_queue, ships):
        Thread.__init__(self)
        self.borneinf = borneinf
        self.bornesup = bornesup
        self.queue = queue
        self.time_queue = time_queue
        self.ships = ships
        self.id = id
        print(f'Starting Thread {id}')

    def run(self):
        liste = np.arange(self.borneinf, self.bornesup+1, 1, dtype=int)
        listes = np.array_split(liste, 10)
        i = 1
        for elem in listes:
          try:
              get_data(elem[0], elem[-1], self.queue, self.time_queue, self.ships)
              print(f'Thread {self.id} : {i*10}%')
          except:
              print(f'Thread {self.id} : echec {i*10}%')
              traceback.print_exc()
          i += 1



In [ ]:
NB_THREADS = 8

bucket = Queue()
time_bucket = Queue()
liste = np.arange(101384-20000, 101384+1, 1, dtype=int)
listes = np.array_split(liste, NB_THREADS)

thread_list = []
timea = time.time() 
for x in range(NB_THREADS):
    worker = DataGetter(x, listes[x][0], listes[x][-1], bucket, time_bucket, ships)
    worker.start()
    thread_list.append(worker)

for thr in thread_list:
    thr.join()

print(time.time() - timea)

Starting Thread 0
Starting Thread 1
Starting Thread 2
Starting Thread 3
Starting Thread 4
Starting Thread 5
Starting Thread 6
Starting Thread 7
Thread 2 : 10%
Thread 0 : 10%
Thread 3 : 10%
Thread 5 : 10%
Thread 7 : 10%
Thread 1 : 10%
Thread 6 : 10%
Thread 4 : 10%
Thread 4 : 20%
Thread 3 : 20%
Thread 0 : 20%
Thread 2 : 20%
Thread 7 : 20%
Thread 5 : 20%
Thread 1 : 20%
Thread 6 : 20%
Thread 4 : 30%
Thread 2 : 30%
Thread 0 : 30%
Thread 3 : 30%
Thread 1 : 30%
Thread 6 : 30%
Thread 5 : 30%
Thread 7 : 30%
Thread 4 : 40%
Thread 3 : 40%
Thread 0 : 40%
Thread 1 : 40%
Thread 6 : 40%
Thread 2 : 40%
Thread 5 : 40%
Thread 7 : 40%
Thread 3 : 50%
Thread 4 : 50%
Thread 2 : 50%
Thread 0 : 50%
Thread 1 : 50%
Thread 5 : 50%
Thread 7 : 50%
Thread 6 : 50%
Thread 4 : 60%
Thread 0 : 60%
Thread 3 : 60%
Thread 5 : 60%
Thread 2 : 60%
Thread 7 : 60%
Thread 1 : 60%
Thread 6 : 60%
Thread 4 : 70%
Thread 3 : 70%
Thread 0 : 70%
Thread 5 : 70%
Thread 2 : 70%
Thread 7 : 70%
Thread 1 : 70%
Thread 6 : 70%
Thread 0 : 80%
T

**Statistics :**
- 1 thread : ~3s / replay
- 2 threads : 1.3s / replay
- 4 threads : 0.73s / replay
- 8 threads : 0.3s / replay


## Extract from the queue

In [ ]:
bucket.qsize()

80

In [ ]:
l = []
for i in range(bucket.qsize()):
  l.append(bucket.get())

In [ ]:
np.save("/content/drive/My Drive/my_saved_queue2.obj", np.array(l))

In [ ]:
l[0].shape

(194, 877)

In [ ]:
data_train = np.empty((0,877))
for elem in l:
  data_train = np.vstack((data_train, elem))

In [ ]:
np.save("/content/drive/My Drive/data_train2", data_train)

In [ ]:
data_train.shape

(16098, 877)

In [ ]:
test = np.load("/content/drive/My Drive/data_train2.npy", allow_pickle=True) 

In [ ]:
test.shape

(16098, 877)

Fast checking ok